# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [5]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    #Create graph with sink and source
    graph_2, total_demand, total_supply = add_source_sink(graph)
    #Run only when demand constraint run
    if total_demand == total_supply:
        #Run flow algorithm (FF)
        flow_value, flow_path = nx.maximum_flow(graph_2, 'Source', 'Sink')
        #Extract Source and Sink nodes from dictionary
        flow_path = {node: flow_path[node] for node in flow_path if (node != 'Sink' and node!=  'Source')}
        for node in flow_path.keys():
            flow_path[node] = {node2: flow_path[node][node2] for node2 in flow_path[node] if node2 != 'Sink' and node2!=  'Source'}
        return flow_path
    #Else raise error
    else:
        raise nx.NetworkXUnfeasible('Demand constraint not met')

        
def add_source_sink(graph):
    """adds a sink to the graph wth edge for all nodes with demand > 0 and source node for all with edge for all nodes with
        demand < 0 (Ca).
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        An updated graph with a sink and source node that allows FF algorithm to run
        
    """
    graph2 = graph.copy()   
    total_demand = 0
    total_supply = 0
    for state in list(graph2.nodes()):
        #Positive demand states (All - 'CA')
        if graph2.nodes[state]['demand'] > 0:
            graph2.add_edge(state, 'Sink')
            graph2.edges[state, 'Sink']['capacity'] = graph2.nodes[state]['demand']
            total_demand += graph2.nodes[state]['demand']
        #Negative demand states ('CA')
        elif graph2.nodes[state]['demand'] < 0:
            graph2.add_edge('Source', state)
            graph2.edges['Source', state]['capacity'] = - graph2.nodes[state]['demand']
            total_supply -= graph2.nodes[state]['demand']
    return (graph2, total_demand, total_supply)

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [6]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    #Create new dictionary
    nodes = {}
    for node in flow:
        nodes[node] = 0 
    #Fill with appropriate flow
    for outgoing_node in flow:
        for incoming_node in flow[outgoing_node]:
            nodes[outgoing_node] -= flow[outgoing_node][incoming_node]
            nodes[incoming_node] += flow[outgoing_node][incoming_node]
    return nodes

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [8]:
flow = flow_with_demands(G)
div = divergence(flow)
#print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))

Flow satisfies all demands: True
